In [35]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from copy import deepcopy
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
# pn.extension()
pn.extension('tabulator') # for dataframes


# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
# from neuropy.core.session.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSssion

from neuropy.core.session.data_session_loader import DataSessionLoader
from neuropy.core.session.dataSession import DataSession
from neuropy.core.epoch import Epoch
from neuropy.core.epoch import NamedTimerange
from neuropy.core import Laps
from neuropy.core import Position
from neuropy.core import FlattenedSpiketrains
from neuropy.core import Neurons
from neuropy.utils.misc import print_seconds_human_readable
from neuropy.plotting import plot_raster
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.analyses.laps import estimate_laps, compute_laps_spike_indicies
from neuropy.analyses.pho_custom_placefields import PfND
from neuropy.plotting.placemaps import plot_all_placefields

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import PrettyPrintable, get_arguments_as_optional_dict
from PhoPositionalData.debug_helpers import debug_print_spike_counts

# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.plotting.laps import plot_laps_2d
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences
from PhoPositionalData.debug_helpers import *

from PendingNotebookCode import compute_placefields_as_needed, build_configs, build_units_colormap, build_placefield_multiplotter, process_by_good_placefields, estimation_session_laps, compute_placefields_masked_by_epochs, perform_compute_placefields

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
## Bapun Format:
# basedir = '/media/share/data/Bapun/Day5TwoNovel' # Linux
basedir = Path('R:\data\Bapun\Day5TwoNovel') # Windows
# basedir = '/Volumes/iNeo/Data/Bapun/Day5TwoNovel' # MacOS

sess = DataSessionLoader.bapun_data_session(basedir)
active_sess_config = sess.config
session_name = sess.name

# sess.spikes_df.size # (1014937, 18)
# np.sum(sess.neurons.n_spikes) # 1014937
print('session dataframe spikes: {}\nsession.neurons.n_spikes summed: {}\n'.format(sess.spikes_df.shape, np.sum(sess.neurons.n_spikes)))

## Estimate the Session's Laps data using my algorithm from the loaded position data.
# sess = estimation_session_laps(sess)

# print(sess.epochs.to_dataframe())
# print(sess.position.to_dataframe()) # max is in 4.2305 x 10^4
# print(sess.flattened_spiketrains.spikes_df) # t_seconds is in the same units as the position dataframe, which is NOT seconds, and 't' is a factor of 1x10^4 larger than 't_seconds'

# sess.epochs.to_dataframe()
# sess.spikes


sess.panel_dataframes_overview()

# # tabs = pn.Tabs(('epochs',epochs_df_widget), ('laps',laps_df_widget), ('neurons', neurons_plot), dynamic=False)
# tabs = pn.Tabs(tabs_list, dynamic=False)
# tabs

	 basepath: R:\data\Bapun\Day5TwoNovel
	 session_name: RatS-Day5TwoNovel-2020-12-04_07-55-09
converting neuron_type strings to core.neurons.NeuronType objects...
	 done.
Issue with pickled POSIX_PATH on windows for path R:\data\Bapun\Day5TwoNovel\RatS-Day5TwoNovel-2020-12-04_07-55-09.probegroup.npy, falling back to non-pickled version...
Position falling back to legacy loading protocol...: dict_rep: {'traces': array([[-50.946354, -50.946354, -50.946354, ..., -45.350155, -45.350155,
        -45.350155],
       [-50.646282, -50.646282, -50.646282, ...,  64.150757,  64.150757,
         64.150757],
       [  7.124897,   7.124897,   7.124897, ...,   5.571329,   5.571329,
          5.571329]]), 'computed_traces': array([[nan, nan, nan, ..., nan, nan, nan]]), 't_start': 0, 'sampling_rate': 60, 'metadata': None}
linearized position loaded from file.
Loading success: .flattened.spikes.npy.
Loading success: .ripple.npy.
Loading success: .mua.npy.
Loading success: .pbe.npy.
session dataframe spik

Column(min_width=1000, sizing_mode='stretch_width', width_policy='max')
    [0] Row
        [0] Column
            [0] StaticText(name='epochs', value='Epochs 5 rows x 4 cols')
            [1] Tabulator(disabled=True, value=   start   stop  ..., width_policy='min')
        [1] Spacer(width=8)
        [2] Column
            [0] StaticText(name='position', value='Position 2538347 r...)
            [1] Tabulator(disabled=True, name='Position', pagination='remote', value=              ..., width_policy='min')
    [1] Column
        [0] StaticText(name='spikes_df', value='Spikes 16318817 r...)
        [1] Tabulator(disabled=True, name='spikes_df', pagination='remote', value=          flat_spike_idx  ...)

In [ ]:
## Neuron Filtering Performance Testing:
def test_get_by_id(sess, ids):
    copy_sess = deepcopy(sess)
    copy_sess.neurons = copy_sess.neurons.get_by_id(ids)
    copy_sess.flattened_spiketrains = copy_sess.flattened_spiketrains.get_by_id(ids)
    return copy_sess

def test_get_neuron_type(query_neuron_type):
    """ filters self by the specified query_neuron_type, only returning neurons that match. """
    print('Constraining to units with type: {}'.format(query_neuron_type))
    # make a copy of self:
    copy_sess = DataSession.from_dict(deepcopy(sess.to_dict()))
    # update the copy_session's neurons objects
    copy_sess.neurons = copy_sess.neurons.get_neuron_type(query_neuron_type) # active_epoch_session_Neurons: Filter by pyramidal cells only, returns a core.
    copy_sess.flattened_spiketrains = copy_sess.flattened_spiketrains.get_neuron_type(query_neuron_type) # active_epoch_session_Neurons: Filter by pyramidal cells only, returns a core.
    return copy_sess

def test_get_by_neuron_type():
    copy_sess = deepcopy(sess)
    return copy_sess.filtered_by_neuron_type('pyramidal')

# %prun test_get_by_id(sess, np.array([0, 1]))

%prun -s cumulative estimate_pi()

In [ ]:
# sess.epochs
# active_epoch = sess.epochs.get_named_timerange('maze1') # gets a NamedTimerange object
active_epoch = sess.epochs.label_slice('maze2') # gets an Epoch object

# active_epoch = sess.epochs.label_slice(['maze1','maze2']) # gets an Epoch object

# print('active_epoch: {}'.format(active_epoch))
active_epoch

# epochs = active_epoch
# epochs.t_start
# epochs.t_stop
# _prev_time_variable_name = spk_df.spikes.time_variable_name # 't_rel_seconds'
# _prev_time_variable_name

In [26]:
## Efficiently filter by cell type and desired ids

from neuropy.core.neurons import NeuronType

# sess.neuron_ids

def batch_filter_session(sess, position, spikes_df, epochs):
    """ a workaround to efficiently filter DataSession objects by epochs and cell_type (currently hardcoded Pyramidal) that works around the issue with deepcopy(...) on DataSessions filled with Bapun's data. """ 
    position.compute_higher_order_derivatives()
    pos_df = position.compute_smoothed_position_info() ## Smooth the velocity curve to apply meaningful logic to it
    pos_df = position.to_dataframe().copy()
    pos_df.position.speed
    # pos_df = position.to_dataframe().copy() # 159 ms ± 3.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
    spk_df = spikes_df.copy() # 949 ms ± 62.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    spk_df.spikes.set_time_variable_name('t_seconds')
    # spk_df.spikes.time_variable_name # 't_seconds'
    # print(f'spk_df.spikes.time_variable_name: {spk_df.spikes.time_variable_name}')

    # filtering:

    # Filter by multiple conditions:
    # Perform general pre-filtering for neuron type and constraining between the ranges:
    # included_neuron_type = 'pyramidal'
    included_neuron_type = NeuronType.PYRAMIDAL
    filtered_spikes_df = spk_df.query("@epochs.t_start <= `t_seconds` <= @epochs.t_stop and `cell_type` == @included_neuron_type" ) # 272 ms, 393 ms, Wall time: 183 ms
    # filtered_spikes_df = spk_df.query("@epochs.t_start <= `t_seconds` <= @epochs.t_stop and `aclu` in @filtered_spikes_df.spikes.neuron_ids") # 272 ms, 393 ms, Wall time: 183 ms
    # filtered_spikes_df = spk_df.query("@epochs.t_start <= `t_seconds` <= @epochs.t_stop and `aclu` in @spk_df.spikes.neuron_ids and `cell_type` == @included_neuron_type" ) # 272 ms, 393 ms, Wall time: 183 ms
    # filtered_spikes_df = spk_df.query("@epochs.t_start <= `t_seconds` <= @epochs.t_stop and `aclu` in @filtered_spikes_df.spikes.neuron_ids and `cell_type` == @included_neuron_type" ) # 272 ms, 393 ms, Wall time: 183 ms
    # filtered_spikes_df = spk_df.query("`t_seconds` >= @epochs.starts and `t_seconds` <= @epochs.stops")
    # filtered_spikes_df = spk_df.query("`t_seconds` >= @epochs.starts and `t_seconds` <= @epochs.stops and `aclu` in @filtered_spikes_df.spikes.neuron_ids")

    filtered_spikes_df = filtered_spikes_df.spikes.time_sliced(epochs.starts, epochs.stops) # 168 ms ± 4.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each), Wall time: 20.4 ms
    filtered_pos_df = pos_df.position.time_sliced(epochs.starts, epochs.stops) # 27.6 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

    ## Old Separate way:
    # if epochs is not None:
    #     # filter the spikes_df:
    #     filtered_spikes_df = spk_df.spikes.time_sliced(epochs.starts, epochs.stops)
    #     # filter the pos_df:
    #     filtered_pos_df = pos_df.position.time_sliced(epochs.starts, epochs.stops) # 5378 rows × 18 columns
    # else:
    #     # if no epochs filtering, set the filtered objects to be sliced by the available range of the position data (given by position.t_start, position.t_stop)
    #     filtered_spikes_df = spk_df.spikes.time_sliced(position.t_start, position.t_stop)
    #     filtered_pos_df = pos_df.position.time_sliced(position.t_start, position.t_stop)

    # Debug print output:
    print(f'Spikes Dataframe: \nnp.shape(sess.spikes_df): {np.shape(sess.spikes_df)}') # (16318817, 10)
    print(f'np.shape(filtered_spikes_df): {np.shape(filtered_spikes_df)}') # (1236690, 10)

    print(f'Position Dataframe: \nnp.shape(sess.position.to_dataframe()): {np.shape(sess.position.to_dataframe())}') # (2538347, 12)
    print(f'np.shape(filtered_pos_df):{np.shape(filtered_pos_df)}') # (174000, 5)

    # Once filtering is done, apply the grouping:
    Neurons.initialize_missing_spikes_df_columns(filtered_spikes_df)

    # Group by the aclu (cluster indicator) column
    cell_grouped_spikes_df = filtered_spikes_df.groupby(['aclu']) # Wall time: 15.6 ms
    cell_spikes_dfs = [cell_grouped_spikes_df.get_group(a_neuron_id) for a_neuron_id in filtered_spikes_df.spikes.neuron_ids] # a list of dataframes for each neuron_id, Wall time: 723 ms

    # Finally, make the filtered session:
    # neurons_obj = Neurons.from_dataframe(filtered_spikes_df, sess.recinfo.dat_sampling_rate, time_variable_name=spk_df.spikes.time_variable_name)
    neurons_obj = None # Wait, it doesn't even set a neurons object and yet it all still works!!
    # Doesn't mess with laps, probegroup
    filtered_sess = DataSession(sess.config, filePrefix=sess.filePrefix, recinfo=sess.recinfo, eegfile=sess.eegfile, datfile=sess.datfile, neurons=neurons_obj, probegroup=sess.probegroup, position=Position(filtered_pos_df, metadata=sess.position.metadata), paradigm=epochs, ripple=sess.ripple, mua=sess.mua, laps=sess.laps, flattened_spiketrains=FlattenedSpiketrains(filtered_spikes_df, time_variable_name=spk_df.spikes.time_variable_name, t_start=epochs.t_start, metadata=sess.flattened_spiketrains.metadata)) # 15.6 ms

    return filtered_sess

# position = sess.position
# spikes_df = sess.spikes_df

# filtered_sess
# print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)

# Filter Session by Epoch:

In [27]:
# sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
# sess.epochs.to_dataframe()
# active_epoch = sess.epochs.get_named_timerange('maze1')
# print('active_epoch: {}'.format(active_epoch))
active_epoch = sess.epochs.get_named_timerange('maze2')
# active_epoch = NamedTimerange(name='maze', start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]])
active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right
active_config = build_configs(active_sess_config, active_epoch, active_subplots_shape = active_subplots_shape)

In [30]:
## All Spikes:
# active_epoch_session = sess.filtered_by_epoch(active_epoch) # old
active_epoch_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())


# position_obj = sess.position
# # position_obj.dt
# position_obj.compute_higher_order_derivatives()
# pos_df = position_obj.compute_smoothed_position_info(N=N) ## Smooth the velocity curve to apply meaningful logic to it
# pos_df = position_obj.to_dataframe()


# active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
# print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
# print(sess.neurons.n_spikes)

# active_epoch_session.spikes_df

# # ## Lap_specific Spikes Only:
# active_lap_specific_epoch_session = lap_specific_session.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
# print_subsession_neuron_differences(lap_specific_session.neurons, active_lap_specific_epoch_session.neurons)
# # print(active_lap_specific_epoch_session.neurons.n_spikes)

## Configure Placefield Calc:
should_display_2D_plots = False
should_save_to_disk = False


# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2, smooth=0.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=10, smooth=0.5, frate_thresh=2.0) # works well
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=2.5, smooth=1.5, frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.5, 0.5), frate_thresh=0.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(3, 4), smooth=(2, 1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 10), smooth=(0.5, 0.5), frate_thresh=2.0) ## Works well for 2D Placemaps
# height: 20.0
# width: 250.0
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 0.2), smooth=(0.5, 0.5), frate_thresh=2.0) ## Extremely Slow
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0, grid_bin=(2.0, 1.0), smooth=(0.5, 0.5), frate_thresh=2.0) ## Very slow, doesn't work

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(0.1, 0.1), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=1, grid_bin=(10, 3), smooth=(1.0, 10.0), frate_thresh=2.0)

# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(25, 9), smooth=(0.0, 0.0), frate_thresh=2.0)
# active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=2.0)
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1) # TODO: FIXME: BUG: when frate_thresh=0.0, there are 0 good placefield_neuronIDs for all computations!


# active_config.computation_config.frate_thresh = 0.0
active_epoch_session.panel_dataframes_overview()

C:\Users\Pho\repos\NeuroPy\neuropy\core\position.py:23: UserWarning: registration of accessor <class 'neuropy.core.position.PositionAccessor'> under name 'position' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PositionAccessor(TimeSlicedMixin):


	 time variable changed!
Spikes Dataframe: 
np.shape(sess.spikes_df): (16318817, 10)
np.shape(filtered_spikes_df): (105840, 10)
Position Dataframe: 
np.shape(sess.position.to_dataframe()): (2538347, 21)
np.shape(filtered_pos_df):(194880, 22)
dataframe shank column does not exist. Initializing it to 1s
dataframe qclu column does not exist. Initializing it to the same as aclu
dataframe cluster column does not exist. Initializing it to the same as aclu


Column(min_width=1000, sizing_mode='stretch_width', width_policy='max')
    [0] Row
        [0] Column
            [0] StaticText(name='epochs', value='Epochs 1 rows x 4 cols')
            [1] Tabulator(disabled=True, value=   start   stop  ..., width_policy='min')
        [1] Spacer(width=8)
        [2] Column
            [0] StaticText(name='position', value='Position 194880 r...)
            [1] Tabulator(disabled=True, name='Position', pagination='remote', value=              ..., width_policy='min')
    [1] Column
        [0] StaticText(name='spikes_df', value='Spikes 105840 r...)
        [1] Tabulator(disabled=True, name='spikes_df', pagination='remote', value=          flat_spike_idx  ...)

# Compute Placefields:

In [31]:
# active_epoch_placefields1D, active_epoch_placefields2D = compute_placefields_masked_by_epochs(active_epoch_session, active_config, included_epochs=None, should_display_2D_plots=should_display_2D_plots) ## This is causing problems due to deepcopy of session.
active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_epoch_session.spikes_df, active_epoch_session.position, active_config.computation_config, None, None, included_epochs=None, should_force_recompute_placefields=True)

Recomputing active_epoch_placefields... 	 done.
Recomputing active_epoch_placefields2D... 	 done.


In [ ]:
# Compare the results

# debug_print_ratemap(active_epoch_placefields1D.ratemap)
# num_spikes_per_spiketrain = np.array([np.shape(a_spk_train)[0] for a_spk_train in active_epoch_placefields1D.spk_t])
# num_spikes_per_spiketrain
# print('placefield_neuronID_spikes: {}; ({} total spikes)'.format(num_spikes_per_spiketrain, np.sum(num_spikes_per_spiketrain)))
debug_print_placefield(active_epoch_placefields1D)

# Get the cell IDs that have a good place field mapping:
active_placefields = deepcopy(active_epoch_placefields2D) # not changed this from the default placefields2D object
good_placefield_neuronIDs = np.array(active_placefields.ratemap.neuron_ids) # in order of ascending ID
good_placefield_tuple_neuronIDs = active_placefields.neuron_extended_ids

## Filter by neurons with good placefields only:
good_placefields_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
good_placefields_session

pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = build_units_colormap(good_placefield_neuronIDs)
active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colormap
active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)

# active_placefields.ratemap.get_sort_indicies()
good_placefield_neuronIDs

In [ ]:
good_placefields_session.panel_dataframes_overview()

In [ ]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_config.plotting_config.active_cells_colormap)

## Main Spike/Placemap plotting:

# Plot Placemaps on 3D Plot

In [ ]:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, active_epoch_session, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter) # [2, 17449]

## Interactive 3D Spike and Behavior Browser: 

In [ ]:
import PhoGui
from PhoGui.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

In [ ]:
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)